In [1]:
%matplotlib inline

In [16]:
#export
from export.nb_02 import *
import scipy
from concurrent.futures import ProcessPoolExecutor

In [6]:
#export
def set_plot_sane_defaults(mode='classic'):
    set_plot_sizes(sml=12, med=14, big=16)
    # see https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
    mpl.style.use({
        'classic': 'default',
        'serious': 'bmh',
        'dark': 'dark_background',
        'boring': 'classic',
        'cool': 'ggplot',
        'seaborn': 'seaborn',
    }[mode])
    mpl.rcParams['figure.facecolor'] = 'white' if mode != 'dark' else 'black'

In [8]:
#export
def set_plot_sizes(sml=12, med=14, big=16):
    plt.rc('font', size=sml)          # controls default text sizes
    plt.rc('axes', titlesize=sml)     # fontsize of the axes title
    plt.rc('axes', labelsize=med)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=sml)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=sml)    # fontsize of the tick labels
    plt.rc('legend', fontsize=sml)    # legend fontsize
    plt.rc('figure', titlesize=big)   # fontsize of the figure title

In [9]:
#export
def set_plot_bgs(color='white'):
    mpl.rcParams['figure.facecolor'] = 'white'

In [10]:
#export
def plot_heatmap(df, figsize=(16, 16)):
    fig, ax = plt.subplots(figsize=figsize)
    return sns.heatmap(df.corr(), annot=True, ax=ax)

In [11]:
#export
def plot_dendrogram(df, figsize=(16, 10)):
    corr = np.round(scipy.stats.spearmanr(df).correlation, 4)
    corr_condensed = scipy.cluster.hierarchy.distance.squareform(1 - corr)
    z = scipy.cluster.hierarchy.linkage(corr_condensed, method='average')
    plt.figure(figsize=figsize)
    return scipy.cluster.hierarchy.dendrogram(
        z, labels=df.columns, orientation='left', leaf_font_size=16)

In [13]:
#export
def plot_pairs_corr(df, figsize=(18, 16)):
    axes = pd.plotting.scatter_matrix(df, alpha=0.3, figsize=figsize, diagonal='kde')
    corr = df.corr().values
    for i, j in zip(*np.triu_indices_from(axes, k=1)):
        axes[i, j].annotate("%.3f" % corr[i, j], (0.8, 0.8), xycoords='axes fraction', ha='center', va='center')

In [21]:
#export
def parallel_trees(m, fn, n_jobs=8):
    return list(ProcessPoolExecutor(n_jobs).map(fn, m.estimators_))

In [22]:
#export
def get_ensamble_preds(model, x):
    """Return separately the predictions done by all the estimators in ensemble.
    Parameters
    ----------
    model
    x
    Returns
    -------
    predictions : ndarray shaped (estimators, n_samples)
    """
    # TODO: figure out how to get the faster version commented below working
    # (AttributeError: Can't pickle local object 'get_ensamble_preds.<locals>.get_preds')
    # def get_preds(t):
    #     return t.predict(x)
    #
    # return np.stack(parallel_trees(model, get_preds))

    # WORKAROUND for large datasets:
    # run code above manually (eg. in script.notebook), then use
    # make_ensemble_preds_with_confidence_table directly instead of get_ensemble_preds_with_confidence

    return np.stack([t.predict(x) for t in model.estimators_])

In [14]:
#export
def show_cat_feature_vs_y(df, fld, y_fld, id_fld=None):
    pivot_args = dict(
        data=df, index=fld, columns=y_fld,
        aggfunc='size', fill_value=0,
    )
    tbl_args = pivot_args.copy()
    if id_fld:  # explicit id field is required to show "All" in table
        tbl_args.update(aggfunc='count', values=id_fld, margins=True)
    tbl = pd.pivot_table(**tbl_args)
    print(tbl)
    plot_tbl = pd.pivot_table(**pivot_args)
    plot_tbl.plot.bar()

In [17]:
#export
def make_ensemble_preds_with_confidence_table(df_raw_val, preds, fld, y_fld):
    df = df_raw_val.copy()

    df['pred_std'] = np.std(preds, axis=0)
    df['pred'] = np.mean(preds, axis=0)

    flds = [fld, y_fld, 'pred', 'pred_std']
    tbl = df[flds].groupby(flds[0], as_index=False).mean()

    return tbl

In [18]:
#export
def get_ensemble_preds_with_confidence(
        model, x_val, df_raw_val, fld, y_fld
):
    preds = get_ensamble_preds(model, x_val)
    return make_ensemble_preds_with_confidence_table(df_raw_val, preds, fld, y_fld)

In [19]:
#export
def rf_feat_importance(model, df):
    return pd.DataFrame(
        {'cols': df.columns, 'imp': model.feature_importances_}
    ).sort_values('imp', ascending=False)

In [20]:
#export
def rf_show_plot_fi(model, df, figsize=(12,7)):
    fi = rf_feat_importance(model, df)
    print(fi)
    fi.plot('cols', 'imp', 'barh', figsize=figsize, legend=False)